In [ ]:
import scipy.io
from tqdm import tqdm
from typing import List, Dict, Any
import json
import numpy as np
OpenPoseMap = [
    "Nose",
    "Neck",
    "RShoulder",
    "RElbow",
    "RWrist",
    "LShoulder",
    "LElbow",
    "LWrist",
    "MidHip",
    "RHip",
    "RKnee",
    "RAnkle",
    "LHip",
    "LKnee",
    "LAnkle",
    "REye",
    "LEye",
    "REar",
    "LEar",
    "LBigToe",
    "LSmallToe",
    "LHeel",
    "RBigToe",
    "RSmallToe",
    "RHeel",
    "Background",
]

In [ ]:
bones = [[0,1],   #        Nose - Neck
[1,2],   #        Neck -> Right Shoulder
[2,3],   #        Right Shoulder -> Right Elbow
[3,4],   #        Right Elbow -> Right Wrist
[1,5],   #        Neck -> Left Shoulder
[5,6],   #        Left Shoulder -> Left Elbow
[6,7],   #        Left Elbow -> Left Wrist
[1,8],   #        Neck -> Mid Hip
[8,9],   #        Mid hip -> Right Hip
[9,10],   #        Right Hip -> Right Knee
[10,1],   #       Right Knee -> Right Ankle
[8,12],   #        Mid hip -> Left hip
[12,1],   #       left hip -> left knee
[13,1],   #       Left knee -> left ankle
[0,1],   #       nose -> right eye
[0,1],   #       nose -> left eye
[15,17],   #          right eye -> right ear
[16,18],
[17,15], #....
[0,15],
[0,16],
]#          left eye -> left ear


In [ ]:
import os
openposeOutputDirectory = "D:\DystoniaCoalition\OpenPose100\\"
openposeOutputsMain =  os.listdir(openposeOutputDirectory)[1]  #single folder

mp4foldername = os.path.join(openposeOutputDirectory, openposeOutputsMain)
openposeOutputs =  os.listdir(mp4foldername)

mp4toJSON =[]

for idx, filex in enumerate(openposeOutputs):
    data = json.load(open( os.path.join(mp4foldername, filex)))
    peoples_frame= []
    for people in data['people']:
        peopleCoordinate = []
        for i in range(0, 75, 3):
            if not [people['pose_keypoints_2d'][i], people['pose_keypoints_2d'][i+1]]==[0,0]:
                peopleCoordinate.append([people['pose_keypoints_2d'][i], people['pose_keypoints_2d'][i+1], np.floor((i+1)/3)]) 
        peoples_frame.append(peopleCoordinate )
    # if (len(data['people'])!=1):
    #     print(len(data['people']))
    mp4toJSON.append(peoples_frame)

# for i in bones:
#     bone_each.append(i) if all(x in [d['i'] for d in poses[0][0]] for x in i) else None  
# mp4toJSON

In [ ]:
import json 
import matplotlib.pyplot as plt
import numpy as np 

# fig, ax = plt.subplots()
# plt.autoscale(False)

for eachFrame in mp4toJSON[0:5]: #change
    for p in eachFrame:
        fig, ax = plt.subplots()
        ax.set_ylim(ax.get_ylim()[::-1])
        ax.scatter([e[0]  for e in p], [e[1] for e in p] )
        for e in p:
            ax.annotate(e[2], (e[0], e[1]))

        point={}
        for e in p:
            point[e[2]]= [e[0], e[1]]
        for i in bones:
            if ((i[0] in point) and (i[1] in point)):
                x = point[i[0]][0], point[i[1]][0]
                y = point[i[0]][1], point[i[1]][1]
                ax.plot(x,y)  
        ax.axis('scaled') 
        ax.set_xlim(0, 640)
        ax.set_ylim((0, 480)[::-1])

In [44]:
import plotly.graph_objects as go
import plotly.tools as tls
import plotly.express as px


# for idv, eachFrame in enumerate(mp4toJSON[0:5]): #change
    # for p in eachFrame:
        # fig, ax = plt.subplots()
        # ax.set_ylim(ax.get_ylim()[::-1])
        # ax.scatter([e[0]  for e in p], [e[1] for e in p] )
fig = px.scatter(x= [e[0] for idx, eachFrame in enumerate(mp4toJSON)  for p in eachFrame  for e in p ], 
                 y=  [e[1]  for idx, eachFrame in enumerate(mp4toJSON)for p in eachFrame  for e in p ], 
                    range_x=[0,640], range_y=[480,0],
                    animation_frame= [idx for idx, eachFrame in enumerate(mp4toJSON)  for p in eachFrame  for e in p ],
                    animation_group= [pidx for idx, eachFrame in enumerate(mp4toJSON)  for p in eachFrame  for pidx in enumerate(p) ],
                     width=640, height=480)
post_script = """
    gd = document.getElementById('{plot_id}');
    gd.on('plotly_click', function(data) {
        console.log(data)
        // <image href="mdn_logo_only_color.png" x="80" y="60" width="480" height="266"></image>
        return false;
    });
"""
    
fig.show()
fig.write_html("plotlydemofile.html", post_script=post_script, full_html=False, include_plotlyjs ='cdn')


        # for e in p:
        #     ax.annotate(e[2], (e[0], e[1]))

        # point={}
        # for e in p:
        #     point[e[2]]= [e[0], e[1]]
        # for i in bones:
        #     if ((i[0] in point) and (i[1] in point)):
        #         x = point[i[0]][0], point[i[1]][0]
        #         y = point[i[0]][1], point[i[1]][1]
        #         ax.plot(x,y)  
        # ax.axis('scaled') 
        # ax.set_xlim(0, 640)
        # ax.set_ylim((0, 480)[::-1])
        # fig.show()
        # plotly_fig = tls.mpl_to_plotly(fig,)
        # plotly_fig.show()
        # plotly_fig.to_html(include_plotlyjs='cdn')